In [1]:
#Inspiration https://www.analyticsvidhya.com/blog/2020/08/web-scraping-selenium-with-python/

!pip install selenium

In [36]:
#Importing packages

In [38]:
import os
import pandas as pd
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException

In [110]:
#https://chromedriver.chromium.org/downloads
#Download latest driver from above location. It should be as per the chrome browser version.

driver = webdriver.Chrome(r'D:\01-Works\00-Doc-to-Sync\0-Download\chromedriver_win32\chromedriver.exe')

In [39]:
#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\92.0.4515.43\chromedriver.exe] found in cache


In [40]:
search_url='https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568'

driver.get(search_url.format(q='Car'))

In [41]:
#Scroll to the end of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)#sleep_between_interactions

In [54]:
#Locate the images to be scraped from the current page 
imgResults = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
totalResults=len(imgResults)

In [55]:
totalResults

60

In [56]:
#Click on each Image to extract its corresponding link to download

img_urls = set()
for i in  range(0,len(imgResults)):
    img=imgResults[i]
    try:
        #img.click()
        #time.sleep(2)
        actual_images = driver.find_elements_by_css_selector('BUooTd')
        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                img_urls.add(actual_image.get_attribute('src'))
    except ElementClickInterceptedException or ElementNotInteractableException as err:
        print(err)

In [57]:
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions

In [58]:
def getImageUrls(name,totalImgs,driver):
    
    search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
    driver.get(search_url.format(q=name))
    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from{results_start}:{totalResults}")
        
        for img in thumbnail_results[results_start:totalResults]:
            
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.Q4LuWd')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.add(actual_image.get_attribute('src'))
            
            img_count=len(img_urls)
            
            if img_count >= totalImgs:
                print(f"Found: {img_count} image links")
                break
            else:
                print("Found:", img_count, "looking for more image links ...")                
                load_more_button = driver.find_element_by_css_selector(".mye4qd")
                driver.execute_script("document.querySelector('.mye4qd').click();")
                results_start = len(thumbnail_results)
    return img_urls

In [59]:
def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
       
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

In [60]:
def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in list(searchNames):
        path=os.path.join(destDir,name)
        if not os.path.isdir(path):
            os.mkdir(path)
        print('Current Path',path)
        totalLinks=getImageUrls(name,totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = f"{i:150}.jpg"
                downloadImages(path,file_name,link)
            
searchNames=['Car','horses'] 
destDir=f'./'
totalImgs=5


In [61]:
saveInDestFolder(searchNames,destDir,totalImgs,driver)

Current Path ./Car
Found: 48 search results. Extracting links from0:48
Found: 12 image links
totalLinks {'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSxLWSSZErE7njyrtBw6t8bvfedGm5DVbYxuCmtegAwm5EHntmwe7qGE3vg8ERVlPqxPE4&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXKe20cPSa-seHBKUmO0c_mi_qLp-gpG10ub-WJoAEv4a_2G9qSK_QNdWAf1vYoZxua3s&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThjrUPUNeM-mmijksHZGX0xNXEHuRtfQ3GRDtq5F3_vtY5hhAK1XVvysSz5lwhKj9tPAQ&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxEcewM4K_Xd9jc1SZaX_Kq2J42zOO8BOHcBG76903VWc0n83uJ6ETJ7vnuKFXbWezvu8&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSs_bsy7VYFosE8JVq6dUPXTYeXHmn6HTDoFWJ8Jy4GdokdWPc0-aA37rOJRw5psFP-Ahw&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRViBCZFQ78mp3Fv-q24BnLBXfBpBTvi7aXaq9rWvfjyNUaXuDZ7uYeooFBnkYdUeGwtpk&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpgFiDjxOtt2_kaTCv7KzThk

SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ9vEzzK3aF_5hH8iVwmgczV0RK5nlXBF5O9c9wj6ztvIm3MemWD3DBVFd3Ov1AD-fQfEM&usqp=CAU - AT: ./horses\                                                                                                                                                     4.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjx3fzYXRUTNAAW5rx1RJ1SKkdQf1yN6-UpG93gPV754Fuy3JrEp5RHG6lIi1KEfdKm5g&usqp=CAU - AT: ./horses\                                                                                                                                                     5.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ105cXoIISnH47hR8PQOSdJV2k57OSU1-2865456QZIe0jUCuf76DNr_rgBQu-GstRSqw&usqp=CAU - AT: ./horses\                                                                                                                                                     6.jpg
SAVED - https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYJsz